In [ ]:
from pathlib import Path
import re
import xlwings as xw
import pandas as pd
import datetime

def get_source(account):
    electronics = [
        "OH_EZPASS_", "HCTRA_", "NY_EZPASS_", "EXPRESS_TOLL_20459648", 'ICD', 'NTTA_',
        "PA_EZPASS_", "EXPRESS_TOLL_13169610", "NJ_EZPASS_", "FASTRAK_", "SUNPASS_",
        "THE_TOLL_ROADS_", "GOOD_TO_GO_", "IPASS_", "TX_TAG_", "SOUTHERN_CONNECTOR_",
        "INDIANA_TOLL_ROADS_", "EXPRESS_TOLL_12871286", "RIVERLINK", "ELECTRONIC TOLLS"
    ]
    
    if account is None:
        return "MAIL TOLL"
    
    account_upper = account.upper()
    for source in electronics:
        if source in account_upper:
            return "ELECTRONIC"
    
    if "CITATION" in account_upper:
        return "CITATION"
    
    return "MAIL TOLL"

source_dir = r'C:/Users/BelindaNamwenge/Documents/Projects/OCR_PROCESSING/scripts'
source_path = Path(source_dir)

excel_files = [file for file in source_path.glob('*.xlsx') if not file.name.startswith('~$')]

if not excel_files:
    print("No Excel files found.")
    exit()

# Create a new workbook
combined_wb = xw.Book()
sheet_counter = 0
merged_df = pd.DataFrame()

for excel_file in excel_files:
    try:
        wb = xw.Book(excel_file)
        for sheet in wb.sheets:
            try:
                data_range = sheet.range('A1').expand().value
                df = pd.DataFrame(data_range[1:], columns=data_range[0])
                file_name = excel_file.name

                week_match = re.search(r'\w+\s+(\d+)', file_name)
                year_match = re.search(r'\((\d{4})\)', file_name)
                report_type_match = re.search(r'(AFP|DSP-MMBT|TRAILERS|TOM)', file_name, re.IGNORECASE)
                week = week_match.group(1) if week_match else ""
                year = year_match.group(1) if year_match else ""
                report_type = report_type_match.group(1).upper() if report_type_match else ""

                df["FileName"] = file_name
                df["WEEK"] = week
                df["YEAR"] = year
                df["REPORT TYPE"] = report_type

                if "TXN PROCESSING DATE" not in df.columns:
                    df["TXN PROCESSING DATE"] = list(map(lambda x, y: datetime.datetime.strptime(f'{y}-W{x}-1', "%Y-W%W-%w") + datetime.timedelta(days=4), df["WEEK"], df["YEAR"]))
                if "SLA Start Date" not in df.columns:
                    df["SLA Start Date"] = df["TXN PROCESSING DATE"].apply(lambda x: x - datetime.timedelta(days=60))
                if "TXN TIME DIFF" not in df.columns:
                    df["TXN TIME DIFF"] = list(map(lambda x, y: (x - y).days, df["TXN PROCESSING DATE"], df["EXIT DATE/TIME"]))
                if ('SOURCE' in df.columns) and not ("ACCOUNT" in df.columns):
                    account_check = df[df["SOURCE"].isna()]
                    for index, row in account_check.iterrows():
                        df.loc[index, "SOURCE"] = row["ACCOUNT"]

                # Add TRANSACTION TYPE logic
                if "TRANSACTION TYPE" not in df.columns:
                    df["TRANSACTION TYPE"] = None

                if all(col in df.columns for col in ["HIGH RATES", "AMOUNT", "TRANSPONDER"]):
                    df.loc[df["HIGH RATES"] > df["AMOUNT"], "TRANSACTION TYPE"] = "Transponder Toll"
                    df.loc[df["AMOUNT"] > df["HIGH RATES"], "TRANSACTION TYPE"] = "Plate Toll"

                    pattern = re.compile(r'[^a-zA-Z0-9]')
                    transponder_null = df["TRANSPONDER"].apply(lambda x: True if pattern.search(str(x)) or x == "null" or x == "-" else False)

                    condition = df["AMOUNT"] == df["HIGH RATES"]
                    df.loc[condition & transponder_null, "TRANSACTION TYPE"] = "Plate Toll"
                    df.loc[condition & ~transponder_null, "TRANSACTION TYPE"] = "Transponder Toll"

                df["SOURCE"] = df["ACCOUNT"].apply(get_source)
                df["SLA MET"] = df["TXN TIME DIFF"].apply(lambda x: "Within SLA" if x <= 60 else "Outside SLA")

                merged_df = pd.concat([merged_df, df], ignore_index=True)
            except Exception as e:
                print(f"Failed to process sheet {sheet.name} in {excel_file}: {e}")
        wb.close()
    except Exception as e:
        print(f"Failed to open workbook {excel_file}: {e}")

merged_df["TXN TIME DIFF"] = list(map(lambda x, y: (x - y).days, merged_df["TXN PROCESSING DATE"], merged_df["EXIT DATE/TIME"]))

# Write the combined data to the sheet in chunks
try:
    while True:
        sheet_counter += 1
        sheet_name = f'Combined Data {sheet_counter}'
        try:
            if sheet_counter > 1:
                combined_sheet = combined_wb.sheets.add(sheet_name)
            else:
                combined_sheet = combined_wb.sheets['Combined Data']

            chunk_size = 100000  # Adjusted chunk size for memory management
            start_row = 1 if sheet_counter == 1 else 2  # Start at 2 for subsequent sheets

            # Write column headers if it's the first sheet
            if sheet_counter == 1:
                combined_sheet.range('A1').value = merged_df.columns.tolist()

            # Calculate the number of rows to write
            end_row = start_row + chunk_size - 1
            rows_to_write = merged_df.iloc[start_row - 1:end_row]

            if rows_to_write.empty:
                break

            try:
                # Write data in chunks
                combined_sheet.range(f'A{start_row}').value = rows_to_write.values.tolist()
                start_row += chunk_size
            except Exception as e:
                print(f"Failed to write data to sheet {sheet_name}: {e}")
        except Exception as e:
            print(f"Failed to create or access sheet {sheet_name}: {e}")

    combined_wb.save('all_sheets.xlsx')

    if len(combined_wb.app.books) == 1:
        combined_wb.app.quit()
    else:
        combined_wb.close()

except Exception as e:
    print(f"An error occurred: {e}")
